In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [7]:
feature_group = feature_store.get_feature_group("time_series_hourly_feature_group", version=1)
print("Feature Group Schema:", feature_group.schema())


[autoreload of hopsworks_common.client.exceptions failed: Traceback (most recent call last):
  File "C:\Users\manas\anaconda3\envs\nyctaxi\Lib\site-packages\IPython\extensions\autoreload.py", line 283, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\manas\anaconda3\envs\nyctaxi\Lib\site-packages\IPython\extensions\autoreload.py", line 508, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\manas\anaconda3\envs\nyctaxi\Lib\site-packages\IPython\extensions\autoreload.py", line 405, in update_generic
    update(a, b)
  File "C:\Users\manas\anaconda3\envs\nyctaxi\Lib\site-packages\IPython\extensions\autoreload.py", line 357, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\manas\anaconda3\envs\nyctaxi\Lib\site-packages\IPython\extensions\autoreload.py", line 405, in update_generic
    update(a, b)
  File "C:\Users\manas\anaconda3\envs\nyctaxi\Lib\site-packages\IPython\extensions\autore

Exception: Couldn't find client. Try reconnecting to Hopsworks.

In [6]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-05 05:40:41,310 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 05:40:41,319 INFO: Initializing external client
2025-03-05 05:40:41,320 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 05:40:42,510 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215655
Fetching data from 2025-02-04 00:10:41.310792+00:00 to 2025-03-04 23:10:41.310792+00:00


ValueError: Cannot generate dataset(s) from the given start/end time because event time column is not available in the left feature groups. A start/end time should not be provided as parameters.

In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

NameError: name 'ts_data' is not defined

In [ ]:
ts_data.info()

In [ ]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [ ]:
ts_data.info()

In [ ]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [ ]:
features

In [ ]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

In [ ]:
current_date

In [ ]:
features

In [ ]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

In [ ]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [ ]:
predictions

In [ ]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values